In [ ]:
from pathlib import Path
import h5py
from typing import Tuple, Dict
import numpy as np

SIM_PATH_LEGACY = Path("../data/results/salida_modelo.h5")
print(SIM_PATH_LEGACY.resolve())  # Te muestra la ruta absoluta
print(SIM_PATH_LEGACY.exists())  # True si existe, False si no

In [8]:
def load_all_datasets(path: str | Path) -> Dict[str, np.ndarray]:
    """Carga todos los datasets de un archivo HDF5 plano (sin grupos)."""
    arrays: Dict[str, np.ndarray] = {}
    with h5py.File(path, "r") as f:
        for name, ds in f.items():
            if isinstance(ds, h5py.Dataset):
                arrays[name] = ds[:]  # pasar a numpy
    return arrays

In [ ]:
# Abrir archivo en modo lectura
with h5py.File(SIM_PATH_LEGACY.resolve(), "r") as f:
    # Listar los grupos principales
    for key in f.keys():
        print(key)
        print(f[key])

In [9]:
data = load_all_datasets(SIM_PATH_LEGACY.resolve())

In [10]:
print(data.keys())

dict_keys(['EUR', 'aguaUtilDisp1', 'aguaUtilDisp2', 'aguaUtilDisp3', 'aguaUtilDisp4', 'biomasaAcumulada', 'biomasaDiaria', 'cEstresHidrico', 'cEstresHidricoRadiacion', 'cEstrsTerminco', 'ceh_pc', 'diasDesdePrec', 'drenaje', 'evaporacion', 'ici', 'percolacion_capa_1', 'percolacion_capa_2', 'percolacion_capa_3', 'porcAguaUtil', 'porcCobertura', 'precipitacion_efectiva', 'profRaiz', 'rendimiento', 'transpiracion'])


In [ ]:
# evaporacion
print(data["evaporacion"][:20, 0, 0])
# transpiracion
print(data["transpiracion"][:20, 0, 0])

[0.         1.52686843 2.11542305 2.64202703 2.21356327 0.77935712
 0.27125993 2.06643849 1.4433079  0.89174584 2.41589742 2.33516053
 2.82796474 2.87218458 2.82526437 3.20832561 2.57350085 2.76655435
 1.60564981 2.58212709]


In [37]:
print(data["evaporacion"][4, :5, :5].T)

[[2.21356327 2.21356327 2.21356327 2.21356327 2.21356327]
 [0.98993559 2.21356327 2.21356327 2.21356327 2.21356327]
 [0.98993559 0.98993559 2.21356327 2.21356327 2.21356327]
 [0.98993559 0.98993559 0.98993559 0.98993559 0.98993559]
 [0.98993559 0.98993559 0.98993559 0.98993559 2.21356327]]


In [27]:
# par
print(data["EUR"][:20, 0, 2])
print(data["EUR"][:20, 0, 0])

[3.39047619 3.21904762 3.33333333 3.63809524 3.88571429 3.08571429
 2.34285714 3.2        2.99047619 2.66666667 3.35238095 3.08571429
 3.58095238 3.6        3.92380952 4.         4.         4.
 4.         4.        ]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]


In [15]:
print(data["precipitacion_efectiva"][:20, 0, 0])

[ 0.      0.      0.      0.      0.      0.      8.93    0.     10.8585
  0.      0.      0.      0.      0.      0.      0.      0.      0.722
  1.691   0.    ]
